In [4]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from skimage.io import imread, imshow,imsave
from skimage.color import rgb2hsv
from PIL import Image
from skimage.util import img_as_ubyte

In [9]:
def func1(path,imgname,clsid):
    img2 = cv2.imread(path, cv2.IMREAD_COLOR)
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    _,threshold = cv2.threshold(img, 10, 255, 
                                cv2.THRESH_BINARY)
    contours,_=cv2.findContours(threshold, cv2.RETR_TREE,
                                cv2.CHAIN_APPROX_SIMPLE)

    ans = []
    for cnt in contours :
        area = cv2.contourArea(cnt)
        if area > 5000: 
            approx = cv2.approxPolyDP(cnt, 0.0005 * cv2.arcLength(cnt, True), True)
            x,y,w,h = cv2.boundingRect(cnt)
            a1 = ((x+w)/2)/512
            b1 = ((y+h)/2)/512
            c1 = w/512
            d1 = h/512
            ans.append("{} {} {} {} {}\n".format(clsid,a1,b1,c1,d1))
    file = open("D:\\Final Yr Project\\YOLO\\labels\\val\\{}.txt".format(imgname),"w")
    file.writelines(ans)
    file.close()

In [10]:
def func(path,imgname,clsid):
    crop=cv2.imread(path)
    crop_hsv = rgb2hsv(crop)
    lower_mask = crop_hsv[:,:,0] > 0.09
    #refer to hue channel (in the colorbar)
    upper_mask = crop_hsv[:,:,0] < 0.45
    #refer to transparency channel (in the colorbar)
    saturation_mask = crop_hsv[:,:,1] > 0.4

    mask = upper_mask*lower_mask*saturation_mask
    red = crop[:,:,0]*mask
    green = crop[:,:,1]*mask
    blue = crop[:,:,2]*mask
    crop_masked = np.dstack((red,green,blue))
    imsave("D:\\Final Yr Project\\data\\test.jpg",crop_masked)
    func1("D:\\Final Yr Project\\data\\test.jpg",imgname,clsid)

In [11]:
folder_dir = "D:\\Final Yr Project\\YOLO\\images\\val"
clsid = 0
for fold in os.listdir(folder_dir):
    for images in os.listdir(folder_dir+"\\"+fold):
        if (images.endswith(".jpg")):
            path = folder_dir+"\\"+fold+"\\"+images
            func(path,images[:-4],clsid)
    clsid+=1